In [ ]:
pip install -q yahoo_fin

In [ ]:
pip install --upgrade yfinance

In [3]:
mkdir -p data results log

In [4]:
from google.colab import drive
drive.mount('/content/drive')
# ! find /content/drive/MyDrive/colab/ -type f -mtime +180 -delete -print


Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd

In [6]:
def show_prediction(name, suffix):
  df = pd.read_csv(f'results/{name}_{suffix}_predictions.csv')
  df['predicted_return'] = ((df['predicted_max']+df['predicted_min']+df['predicted_close'])/3 - df['current_price'])/df['current_price']
  return df.sort_values('predicted_return', ascending=False, key=abs)


In [7]:
def run_command_with_check(command):
  """Runs a command, prints stderr, and raises an exception if it fails."""
  process = subprocess.run(command, shell=True, capture_output=True, text=True, check=False)

  if process.returncode != 0:
    print("stderr:", process.stderr)  # Print stderr only on error
    raise subprocess.CalledProcessError(process.returncode, process.args, output=process.stdout, stderr=process.stderr)
  else:
    # Print or process stdout if needed
    #print("stdout:", process.stdout)
    pass

  return process.returncode


In [8]:
import os
import subprocess
def run_model_nn_cmd(prefix, version, do_train=False, window_size=None, future_days=None, hourly=False, copy_tickers=True):
    # copy drive/MyDrive/colab/results/nnreturn.py to local folder
    tickers = f"{prefix}tickers.csv"
    run_command_with_check(f"cp  /content/drive/MyDrive/colab/nnreturn.py .")
    if copy_tickers:
      run_command_with_check(f"cp  /content/drive/MyDrive/colab/tickers/{tickers} .")
    if not do_train:
      run_command_with_check(f"cp  /content/drive/MyDrive/colab/models/{prefix}_{version}_*.pth .")
      # Run nnreturn.py in results folder and throw exception if error
      today = pd.to_datetime('today').strftime('%m%d')
      run_command_with_check(f"rename 's/{version}/{today}/' *.pth")
      cmd_arg = ""
      if window_size:
        cmd_arg += f" --window-size {window_size}"
      if future_days:
        cmd_arg += f" --future-days {future_days}"
      if hourly:
        cmd_arg += " --hourly"
      run_command_with_check(f"python3 nnreturn.py --tickers-file {prefix}tickers.csv --mode analyze {cmd_arg}")
      run_command_with_check(f"cp {prefix}*predictions.csv results")
      # Get today's date

      return show_prediction(prefix, today)
    else:
      raise Exception("Not implemented")


In [ ]:
run_model_nn_cmd('etf', '0730').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
4,DXYZ,26.29,2025-12-05,2026-01-02,25.09,30.11,30.33,0.084443
18,SPY,685.69,2025-12-05,2026-01-02,649.50,697.55,621.26,-0.043149
2,ARKW,155.82,2025-12-05,2026-01-02,155.06,167.30,163.93,0.040282
31,XME,99.74,2025-12-05,2026-01-02,96.99,110.65,103.26,0.039035
1,ARKK,81.91,2025-12-05,2026-01-02,81.28,89.72,84.07,0.038009
27,XLP,78.46,2025-12-05,2026-01-02,77.27,82.95,81.96,0.028889
25,XLI,154.58,2025-12-05,2026-01-02,145.57,156.42,149.37,-0.026696
0,ARKF,49.99,2025-12-05,2026-01-02,48.59,53.65,51.56,0.025538
20,VNQ,89.88,2025-12-05,2026-01-02,88.10,93.59,94.03,0.022549
3,DIA,480.03,2025-12-05,2026-01-02,458.83,482.24,466.59,-0.022519


In [ ]:
# 0716 is hourly 20
run_model_nn('idx', '0716', window_size=20, future_days=8, hourly=True).head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
23,XLB,89.61,2025-07-17,2025-07-19,89.62,90.66,91.40,0.010601
31,XLV,132.58,2025-07-17,2025-07-19,132.34,134.13,134.68,0.008573
20,URA,40.29,2025-07-17,2025-07-19,39.80,40.24,39.87,-0.007942
4,EEM,49.12,2025-07-17,2025-07-19,48.63,48.96,48.71,-0.007193
9,IXP,110.10,2025-07-17,2025-07-19,109.92,111.52,110.81,0.005904
14,JXI,73.21,2025-07-17,2025-07-19,72.43,73.40,72.53,-0.005782
18,SMH,292.19,2025-07-17,2025-07-19,292.37,295.67,293.22,0.005350
13,JETS,25.00,2025-07-17,2025-07-19,24.92,25.34,25.07,0.004400
11,IWO,293.85,2025-07-17,2025-07-19,292.41,296.41,296.18,0.003914
12,IYZ,29.77,2025-07-17,2025-07-19,29.50,29.77,29.72,-0.003583


In [ ]:
# pip install  git+https://github.com/ahsank/runml#egg=runml

In [ ]:
import runml.pipeline
from runml import pipeline,findata
#from importlib import reload
# reload(runml.pipeline)

In [ ]:
# Skip some days from consecutive window to lessen test /train overlap
# findata.G_NON_OVERLAP= 0

Run following if like to save results in drive

In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    # Simplify following using apply
    def calculate_alloc(row):
        gain_low = min(row['Gain_l'], 0)
        gain_high = max(row['Gain_h'], 0)
        gain = min(row['Gain'], gain_high)
        average_gain = (gain_low+gain_high + 2*gain)/4
        if (average_gain == 0):
          return 0
        avg_accuracy = (row['Accu']+row['Accu_l']+row['Accu_h'])/3
        alloc =  avg_accuracy/stop_loss - (1-avg_accuracy)*profit_factor/abs(average_gain)
        return round(max(alloc, 0), 2)
    df['Alloc'] = df.apply(calculate_alloc, axis=1)
    return df

pd.options.display.max_columns = None

In [ ]:
import os
# findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  # Save datafrme to prefix-version-prediction.csv
  df.to_csv(f"results/{prefix}-{version}-prediction.csv")
  return df



In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax' # Change to minmax
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'BYRN', 'CELH', 'CFLT', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ELF', 'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HCP', 'HIMS', 'HOOD', 'HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JAKK', 'JKS', 'KD', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR', 'MRVL', 'MXL','MTCH', 'MU',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PAR', 'PANW',  'PAYX', 'PD', 'PI', 'PINS',
           'RBLX', 'RKLB', 'RMBS', 'RUM', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'SOUN', 'TEM', 'TEAM', 'TDOC', 'TMDX', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '2a', tickers, True)

In [ ]:
addAllocHL(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'DXYZ',
            'EEM', 'FDN', 'FPX',
            'ICLN', 'IGV', 'IJK', 'IJR', 'IPO',
            'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'JXI', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'URA', 'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '1b', tickers1, True)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

In [ ]:
df1.to_csv('results/etf_12b_predictions.csv')

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax'
tickers3 = [ 'AIRI', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS',  'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CHGG', 'CLOV', 'CRCT', 'CXM',
            'DLO', 'DM', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MITK', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PLUG', 'PTON', 'PUBM',
            'REAL', 'REI', 'RIG', 'RVYL',
            'SABR', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'VTEX', 'VSAT', 'YEXT']
df3 = run_model('vols', '12b', tickers3, False)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'KO', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
df4 = run_model('mcap', '1b', tickers4, True)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CX', 'DBOEY', 'DNNGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FDRVF', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']

df5 = run_model('gstock', '2a', tickers5, False)

In [ ]:
addAllocHL(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.G_SCALER = 'standard'
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'KWT', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '12a', tickers6, False)

In [ ]:
addAllocHL(df6, 0.07, 1)
df6.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'standard'
tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE', 'ASC',
            'BAH', 'BAX', 'BKR', 'BLK', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IFF', 'IP', 'IPI', 'IVZ',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW', 'ZTS']
df7 = run_model('val', '1a', tickers7, False)

In [ ]:
addAllocHL(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers8 = [ 'DBC', 'FXB', 'FXE', 'FXF', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '12a', tickers8, True)

In [ ]:
addAllocHL(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

In [9]:
run_model_nn_cmd('pennystock', '0630').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
60,SLE,0.95,2025-12-09,2026-01-06,0.78,1.52,1.34,0.277193
27,GTE,4.68,2025-12-09,2026-01-06,4.39,6.50,5.41,0.160969
9,AXTI,14.86,2025-12-09,2026-01-06,11.11,15.44,11.45,-0.147600
37,LUMN,8.36,2025-12-09,2026-01-06,6.07,8.68,6.68,-0.145534
11,BKKT,13.40,2025-12-09,2026-01-06,10.29,13.71,10.97,-0.130100
19,CXM,7.91,2025-12-09,2026-01-06,7.56,9.37,9.70,0.122208
13,BLNK,1.01,2025-12-09,2026-01-06,0.78,1.14,0.77,-0.112211
56,RVYL,0.33,2025-12-09,2026-01-06,0.24,0.33,0.31,-0.111111
17,CLOV,2.58,2025-12-09,2026-01-06,2.51,3.06,2.99,0.105943
44,ONDS,9.48,2025-12-09,2026-01-06,7.85,9.77,7.83,-0.105134


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '11b', tickers9, False)

In [ ]:
addAllocHL(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

In [ ]:
run_model_nn_cmd('bluechipstock', '0221').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
35,SBUX,83.45,2025-12-08,2026-01-05,83.65,93.82,88.67,0.063072
27,MMC,182.24,2025-12-08,2026-01-05,180.39,196.68,201.01,0.057360
4,AMZN,226.85,2025-12-08,2026-01-05,202.07,234.40,209.79,-0.050386
31,NOW,853.71,2025-12-08,2026-01-05,852.14,941.62,884.00,0.045538
33,PYPL,61.58,2025-12-08,2026-01-05,60.42,67.77,64.43,0.042655
11,CRM,257.07,2025-12-08,2026-01-05,257.64,273.51,269.83,0.038602
34,QCOM,174.20,2025-12-08,2026-01-05,160.25,175.52,168.15,-0.035744
20,HD,349.75,2025-12-08,2026-01-05,347.67,368.44,369.03,0.034205
23,ISRG,565.03,2025-12-08,2026-01-05,554.49,617.97,577.93,0.032624
1,ACN,266.75,2025-12-08,2026-01-05,262.34,287.60,275.07,0.030940


In [ ]:
run_model_nn_cmd('country', '1112').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
24,XCEM,37.99,2025-12-04,2026-01-01,35.08,38.25,35.44,-0.045626
12,EWT,64.53,2025-12-04,2026-01-01,60.84,65.35,61.16,-0.032233
0,ARGT,94.51,2025-12-04,2026-01-01,91.49,99.16,99.00,0.021585
23,VNM,18.60,2025-12-04,2026-01-01,17.72,19.29,17.82,-0.017384
9,EWC,53.03,2025-12-04,2026-01-01,51.07,53.71,51.55,-0.017349
5,EWA,26.30,2025-12-04,2026-01-01,25.28,26.69,25.64,-0.016350
11,EWP,52.58,2025-12-04,2026-01-01,50.13,53.60,51.80,-0.014010
1,EWD,48.11,2025-12-04,2026-01-01,47.30,48.73,46.47,-0.012679
20,KWT,39.78,2025-12-04,2026-01-01,37.68,40.55,39.63,-0.012402
17,ILF,31.95,2025-12-04,2026-01-01,30.82,32.65,31.44,-0.009807


In [ ]:
run_model_nn_cmd('global', '1107').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
15,DNNGY,7.24,2025-12-02,2025-12-30,5.66,9.15,8.95,0.093923
38,LIN,406.50,2025-12-02,2025-12-30,402.53,444.39,465.68,0.076343
1,ADDDF,185.82,2025-12-01,2025-12-29,180.00,218.07,199.42,0.071808
3,BAK,2.85,2025-12-02,2025-12-30,2.75,3.36,3.05,0.071345
55,SAP,242.07,2025-12-02,2025-12-30,235.24,256.39,280.29,0.062943
9,COTY,3.32,2025-12-02,2025-12-30,3.17,3.86,3.51,0.058233
45,NVO,47.49,2025-12-02,2025-12-30,45.84,53.99,50.71,0.056644
56,SE,137.13,2025-12-02,2025-12-30,134.12,158.67,141.80,0.056394
64,VALE,12.74,2025-12-02,2025-12-30,11.52,13.13,11.52,-0.053637
49,PAX,15.09,2025-12-02,2025-12-30,14.79,16.28,16.53,0.051469


In [ ]:
run_model_nn_cmd('valuestock', '0301').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
63,OLN,19.96,2025-11-12,2025-12-10,19.60,22.78,21.34,0.064128
90,TWI,7.84,2025-11-12,2025-12-10,7.63,8.65,8.57,0.056548
47,HSII,58.81,2025-11-12,2025-12-10,52.91,58.10,57.06,-0.047384
4,ANF,71.28,2025-11-12,2025-12-10,69.53,78.20,76.07,0.046577
67,PETS,2.91,2025-11-12,2025-12-10,2.95,3.15,3.03,0.045819
73,RCL,264.51,2025-11-12,2025-12-10,261.45,286.09,281.79,0.045115
77,SBLK,18.85,2025-11-12,2025-12-10,18.52,21.32,19.20,0.044032
42,FMS,23.93,2025-11-12,2025-12-10,21.68,24.18,22.78,-0.043878
79,SPTN,26.90,2025-09-22,2025-10-20,25.39,27.25,24.55,-0.043494
57,MMM,170.50,2025-11-12,2025-12-10,167.33,185.09,180.54,0.041955


In [ ]:
run_model_nn_cmd('commodity', '0312').head(5)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
8,SLV,45.96,2025-11-14,2025-12-12,44.89,52.83,46.69,0.047360
5,GLD,375.96,2025-11-14,2025-12-12,367.33,428.92,380.72,0.043524
15,WEAT,4.21,2025-11-14,2025-12-12,4.12,4.40,4.27,0.012668
13,UUP,28.11,2025-11-14,2025-12-12,27.48,29.14,28.48,0.009131
4,FXY,59.53,2025-11-14,2025-12-12,58.00,61.63,60.16,0.006719


In [ ]:
run_model_nn_cmd('crypto', '0214').head(5)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
10,BCH-USD,558.16,2025-11-22,2025-12-20,556.97,660.97,605.32,0.088851
8,LTC-USD,82.35,2025-11-22,2025-12-20,73.68,104.66,90.43,0.087917
3,ADA-USD,0.41,2025-11-22,2025-12-20,0.35,0.42,0.36,-0.081301
4,XRP-USD,1.95,2025-11-22,2025-12-20,1.89,2.32,2.08,0.075214
11,ICP-USD,4.05,2025-11-22,2025-12-20,3.43,4.25,3.58,-0.073251


In [ ]:
run_model_nn_cmd('techstock', '0312').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
2,AI,15.27,2025-12-04,2026-01-01,14.68,20.75,17.30,0.151059
6,BILL,52.43,2025-12-04,2026-01-01,52.11,68.23,55.90,0.120478
31,HIMS,40.00,2025-12-04,2026-01-01,33.46,41.24,36.01,-0.077417
50,NVCR,11.62,2025-12-04,2026-01-01,11.41,13.02,12.98,0.073150
7,BMBL,3.73,2025-12-04,2026-01-01,3.18,3.80,3.49,-0.064343
39,KD,26.20,2025-12-04,2026-01-01,24.51,30.25,28.69,0.061705
40,LYFT,22.26,2025-12-04,2026-01-01,20.80,23.51,18.41,-0.060797
76,UI,578.86,2025-12-04,2026-01-01,504.95,615.44,515.64,-0.057901
43,MNDY,154.35,2025-12-04,2026-01-01,148.21,176.23,164.93,0.056841
62,RMBS,99.42,2025-12-04,2026-01-01,87.28,100.52,93.75,-0.056025


In [ ]:
run_model_nn('stock', '0123').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
204,PUBM,12.82,2025-07-03,2025-07-31,12.64,16.44,14.37,0.129745
375,PI,115.86,2025-07-03,2025-07-31,116.02,144.76,128.19,0.119080
17,INTA,48.11,2025-07-03,2025-07-31,45.99,55.07,59.27,0.110857
352,ABR,10.90,2025-07-03,2025-07-31,10.53,12.98,12.60,0.104281
144,LFMD,12.50,2025-07-03,2025-07-31,11.93,14.68,14.55,0.097600
378,RCAT,7.08,2025-07-03,2025-07-31,5.73,7.56,5.93,-0.095104
10,ARQT,13.82,2025-07-03,2025-07-31,13.37,16.73,15.27,0.094308
348,CRM,272.15,2025-07-03,2025-07-31,267.03,319.47,305.57,0.092620
283,NTGR,29.41,2025-07-03,2025-07-31,28.97,35.49,31.90,0.092146
150,POWL,217.66,2025-07-03,2025-07-31,204.27,250.88,256.42,0.089727


In [5]:
import sys
sys.path.append('/content')
# !unzip drive/MyDrive/colab/stock_prediction_new.zip
import stock_prediction.stock_prediction.stock_predictor_lib
from stock_prediction.stock_prediction.stock_predictor_lib import run_model_nn, show_prediction, StockPredictorLib, Config
colab_path = 'content/drive/MyDrive/colab'

In [ ]:
run_model_nn_cmd('buystocks', '0402', False, 500, 250).head(10)
# show_prediction('buystocks', '0508').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
232,SLDP,4.94,2025-12-02,2026-11-17,4.79,9.83,8.73,0.575574
161,EL,98.58,2025-12-02,2026-11-17,98.13,180.11,183.16,0.560154
47,EL,98.58,2025-12-02,2026-11-17,98.13,180.11,183.16,0.560154
330,ENPH,28.86,2025-12-02,2026-11-17,27.62,51.64,53.83,0.537191
331,BABA,160.20,2025-12-02,2026-11-17,152.08,295.59,286.21,0.527008
118,VOD,12.33,2025-12-02,2026-11-17,12.48,23.40,19.98,0.510138
312,RWM,16.60,2025-12-02,2026-11-17,16.19,32.28,26.71,0.509639
96,SLP,20.32,2025-12-02,2026-11-17,18.52,37.42,35.93,0.507054
95,CHGG,0.94,2025-12-02,2026-11-17,0.76,1.78,1.67,0.492908
121,LZ,9.23,2025-12-02,2026-11-17,8.89,16.26,15.98,0.485374


In [ ]:
techcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
colab_path = 'content/drive/MyDrive/colab'
techcfg.model_path = f'/{colab_path}/models/transformer/techstock_1005'
techdf = run_model_nn(f'/{colab_path}/tickers/techstocktickers.csv', cfg=techcfg)

In [ ]:
techdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
51,NTNX,47.11,2025-12-04,2026-01-01,31.10,49.77,33.66,-0.189627
40,LYFT,22.28,2025-12-04,2026-01-01,21.50,30.27,26.47,0.170557
58,PI,159.32,2025-12-04,2026-01-01,106.24,172.22,125.98,-0.153820
8,BYRN,18.95,2025-12-04,2026-01-01,15.23,20.16,13.11,-0.146878
63,SMCI,34.04,2025-12-04,2026-01-01,27.56,34.77,25.22,-0.142675
62,RMBS,99.24,2025-12-04,2026-01-01,72.04,106.35,79.90,-0.132440
45,MPWR,948.28,2025-12-04,2026-01-01,727.25,973.14,771.25,-0.131185
49,NOVT,116.48,2025-12-04,2026-01-01,94.22,118.15,92.82,-0.126631
1,ACLS,89.28,2025-12-04,2026-01-01,76.04,101.46,123.78,0.124851
68,TDOC,7.68,2025-12-04,2026-01-01,5.63,7.87,6.86,-0.116319


In [ ]:
defetfcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
defetfcfg.model_path = f'/{colab_path}/models/rolling/etf_1206'
etfdf = run_model_nn(f'/{colab_path}/tickers/etftickers.csv', cfg=defetfcfg)
etfdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/etf_1206_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/etf_1206_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/etf_1206_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
21,XLB,44.09,2025-12-05,2026-01-02,43.66,49.16,45.91,0.048839
28,XLU,43.30,2025-12-05,2026-01-02,42.63,47.51,45.92,0.047421
31,XME,99.74,2025-12-05,2026-01-02,90.32,100.40,99.31,-0.030713
15,JXI,79.17,2025-12-05,2026-01-02,77.66,85.32,81.27,0.028378
14,JETS,27.43,2025-12-05,2026-01-02,25.43,28.66,26.09,-0.025641
8,IXC,43.23,2025-12-05,2026-01-02,40.94,43.96,41.88,-0.022438
30,XLY,119.73,2025-12-05,2026-01-02,118.14,125.82,122.25,0.019544
4,DXYZ,26.29,2025-12-05,2026-01-02,24.55,27.14,25.66,-0.019272
32,XSW,191.70,2025-12-05,2026-01-02,184.68,202.78,196.71,0.015771
26,XLK,146.60,2025-12-05,2026-01-02,143.89,151.43,151.21,0.015302


In [8]:
tvolcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
tvolcfg.model_path = f'/{colab_path}/models/transformer/pennystock_1210'
tvoldf = run_model_nn(f'/{colab_path}/tickers/pennystocktickers.csv', cfg=tvolcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/pennystock_1210_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/pennystock_1210_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/pennystock_1210_close_model.pth'}


In [9]:
tvoldf[tvoldf['predicted_return']> 0].head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
18,CXM,8.06,2025-12-09,2026-01-06,6.81,17.34,8.48,0.349462
58,SLE,1.02,2025-12-09,2026-01-06,0.80,1.64,1.19,0.186275
50,PTON,6.77,2025-12-09,2026-01-06,5.92,10.38,7.01,0.147710
29,INLX,8.75,2025-12-09,2026-01-06,8.71,9.72,9.80,0.075429
44,OTLY,12.09,2025-12-09,2026-01-06,10.91,13.85,14.15,0.072787
23,GRAB,5.15,2025-12-09,2026-01-06,4.28,6.19,6.05,0.069256
7,AWRE,2.09,2025-12-09,2026-01-06,2.00,2.55,2.12,0.063796
22,GOGO,5.46,2025-12-09,2026-01-06,5.33,6.00,5.90,0.051893
64,TRUP,38.41,2025-12-09,2026-01-06,35.24,43.01,42.91,0.051462
21,FSLY,10.88,2025-12-09,2026-01-06,10.78,11.85,11.64,0.049939


In [ ]:
bctcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
bctcfg.model_path = f'/{colab_path}/models/transformer/bluechipstock_1010'
bctdf = run_model_nn(f'/{colab_path}/tickers/bluechipstocktickers.csv', cfg=bctcfg)
bctdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/bluechipstock_1010_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/bluechipstock_1010_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/bluechipstock_1010_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
3,AMD,217.53,2025-11-28,2025-12-26,199.40,287.08,317.09,0.231355
37,TSLA,430.17,2025-11-28,2025-12-26,320.48,472.67,344.03,-0.118813
21,INTC,40.56,2025-11-28,2025-12-26,31.13,42.14,37.42,-0.090319
15,DELL,133.35,2025-11-28,2025-12-26,106.26,143.03,123.80,-0.067392
41,WMT,110.51,2025-11-28,2025-12-26,94.59,112.03,102.90,-0.066389
9,CAT,575.76,2025-11-28,2025-12-26,480.11,601.42,539.90,-0.061281
38,TXN,168.27,2025-11-28,2025-12-26,154.94,173.72,145.51,-0.060696
39,UNH,329.77,2025-11-28,2025-12-26,280.70,341.08,309.39,-0.058768
2,ADBE,320.13,2025-11-28,2025-12-26,312.50,347.13,346.57,0.047699
22,IBM,308.58,2025-11-28,2025-12-26,268.36,323.14,291.86,-0.045780


In [ ]:
bcrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
bcrcfg.model_path = f'/{colab_path}/models/rolling/bluechipstock_1208'
bcrdf = run_model_nn(f'/{colab_path}/tickers/bluechipstocktickers.csv', cfg=bcrcfg)
bcrdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_1208_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_1208_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_1208_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
4,AMZN,226.97,2025-12-08,2026-01-05,198.64,236.50,214.70,-0.045630
6,BA,205.96,2025-12-08,2026-01-05,189.70,211.89,242.01,0.041626
36,TMO,568.35,2025-12-08,2026-01-05,561.39,623.22,588.24,0.039764
16,DHR,224.22,2025-12-08,2026-01-05,220.64,237.45,240.39,0.038385
34,QCOM,174.03,2025-12-08,2026-01-05,156.34,177.94,168.55,-0.036890
33,PYPL,61.63,2025-12-08,2026-01-05,59.71,66.11,65.83,0.036562
2,ADBE,339.86,2025-12-08,2026-01-05,330.80,366.09,358.46,0.035083
35,SBUX,83.74,2025-12-08,2026-01-05,81.79,91.63,85.50,0.030650
8,BRK-B,495.77,2025-12-08,2026-01-05,482.76,520.51,523.54,0.026558
28,MSFT,489.86,2025-12-08,2026-01-05,465.96,509.54,455.55,-0.026218


In [ ]:
buystkcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
buystkcfg.model_path = f'/{colab_path}/models/transformer/buystocks_1115'
buystkcfg.model.window_size = 500
buystkcfg.model.future_days = 250
buystkdf = run_model_nn(f'/{colab_path}/tickers/buystockstickers.csv', cfg=buystkcfg)

In [ ]:
buystkdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
130,LFMD,4.63,2025-11-14,2026-10-30,3.63,11.99,64.18,4.745140
356,LAES,4.49,2025-11-14,2026-10-30,3.21,39.63,30.40,4.437268
327,OKLO,97.57,2025-11-14,2026-10-30,82.54,984.50,406.78,4.035086
315,INOD,57.21,2025-11-14,2026-10-30,39.89,373.78,401.43,3.749170
345,NUTX,107.11,2025-11-14,2026-10-30,100.94,336.41,954.96,3.332960
95,CHGG,1.09,2025-11-14,2026-10-30,0.58,4.85,8.61,3.293578
329,RGTI,25.48,2025-11-14,2026-10-30,19.92,215.40,82.17,3.153454
330,ACHR,7.88,2025-11-14,2026-10-30,7.71,64.69,23.58,3.060068
258,WKEY,9.20,2025-11-14,2026-10-30,7.15,37.14,65.95,2.994203
352,BULL,8.63,2025-11-14,2026-10-30,4.07,57.04,40.67,2.931248


In [ ]:
stkcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
stkcfg.model_path = f'/{colab_path}/models/transformer/stock_1012'
stkdf = run_model_nn(f'/{colab_path}/tickers/stocktickers.csv', cfg=stkcfg)

In [ ]:
stkdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
335,OKLO,93.87,2025-11-28,2025-12-26,55.75,100.18,66.15,-0.211392
358,BMNR,34.29,2025-11-28,2025-12-26,22.96,36.79,22.94,-0.196170
222,ASTS,56.99,2025-11-28,2025-12-26,40.11,61.74,37.48,-0.185062
218,LEU,261.91,2025-11-28,2025-12-26,161.30,273.59,206.71,-0.183435
26,OSCR,18.03,2025-11-28,2025-12-26,16.94,20.46,26.12,0.174339
42,WGMI,48.71,2025-11-28,2025-12-26,44.74,75.48,51.26,0.173476
341,QBTS,22.70,2025-11-28,2025-12-26,16.68,24.35,15.36,-0.171953
193,U,42.65,2025-11-28,2025-12-26,29.77,45.91,30.97,-0.166471
134,SMCI,33.42,2025-11-28,2025-12-26,20.44,36.14,27.43,-0.162079
251,UEC,12.35,2025-11-28,2025-12-26,9.58,13.08,8.45,-0.160324


In [ ]:
glbcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
glbcfg.model_path = f'/{colab_path}/models/transformer/global_0907'
glbdf = run_model_nn(f'globaltickers.csv', cfg=glbcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/global_0907_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/global_0907_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/global_0907_close_model.pth'}


In [ ]:
glbdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
33,HMC,29.73,2025-11-07,2025-12-05,29.62,34.22,34.07,0.097769
3,BAK,2.44,2025-11-07,2025-12-05,2.45,2.77,2.75,0.088798
38,LIN,422.22,2025-11-07,2025-12-05,420.74,476.39,477.08,0.084908
56,SE,146.78,2025-11-07,2025-12-05,141.42,163.70,169.18,0.077122
66,VNT,37.79,2025-11-07,2025-12-05,37.83,41.91,42.27,0.076211
50,POAHF,40.37,2025-11-06,2025-12-04,40.64,44.84,44.72,0.075056
59,TEF,4.19,2025-11-07,2025-12-05,4.19,4.57,4.74,0.073986
46,NXPI,199.93,2025-11-07,2025-12-05,195.62,219.84,228.47,0.073592
45,NVO,45.17,2025-11-07,2025-12-05,43.24,50.50,50.34,0.063243
35,MELI,2090.31,2025-11-07,2025-12-05,2077.38,2341.55,2241.28,0.062077


In [ ]:
valtcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
valtcfg.model_path = f'/{colab_path}/models/transformer/valuestock_1018'
valtdf = run_model_nn(f'valuestocktickers.csv', cfg=valtcfg)


In [ ]:
valtdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
71,POWL,355.91,2025-11-12,2025-12-10,318.63,406.16,497.69,0.144934
16,CC,12.51,2025-11-12,2025-12-10,11.58,15.90,14.90,0.129230
7,ARE,53.86,2025-11-12,2025-12-10,52.99,59.89,66.73,0.111586
91,UAL,99.94,2025-11-12,2025-12-10,91.43,119.48,121.23,0.107798
14,BXC,57.66,2025-11-12,2025-12-10,55.51,69.67,63.39,0.090126
44,GLW,88.71,2025-11-12,2025-12-10,86.23,108.45,94.80,0.087739
74,RH,160.72,2025-11-12,2025-12-10,156.87,193.12,172.52,0.083686
37,F,13.45,2025-11-12,2025-12-10,12.72,13.75,10.53,-0.083024
58,MOD,147.99,2025-11-12,2025-12-10,142.12,179.25,154.26,0.071311
39,FICO,1778.19,2025-11-12,2025-12-10,1648.44,2256.72,1784.05,0.066480


In [ ]:
commtcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
commtcfg.model_path = f'/{colab_path}/models/transformer/commodity_1021'
commtdf = run_model_nn(f'/{colab_path}/tickers/commoditytickers.csv', cfg=commtcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/commodity_1021_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/commodity_1021_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/commodity_1021_close_model.pth'}


In [ ]:
commtdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
5,GLD,381.15,2025-10-21,2025-11-18,377.94,408.54,416.12,0.051729
12,USO,67.83,2025-10-21,2025-11-18,67.22,69.96,74.23,0.038921
7,PALL,128.90,2025-10-21,2025-11-18,126.66,137.55,134.23,0.030359
15,WEAT,4.04,2025-10-21,2025-11-18,3.87,4.48,4.10,0.027228
0,DBC,22.08,2025-10-21,2025-11-18,20.44,22.28,22.03,-0.022494
8,SLV,47.72,2025-10-20,2025-11-17,46.37,48.63,49.92,0.012294
13,UUP,27.90,2025-10-21,2025-11-18,27.67,28.66,28.09,0.008602
4,FXY,60.58,2025-10-21,2025-11-18,60.38,61.65,60.78,0.005888
1,FXB,128.79,2025-10-21,2025-11-18,127.56,130.69,130.12,0.005176
6,ISHG,75.58,2025-10-21,2025-11-18,74.07,76.31,75.69,-0.002955


In [ ]:
cryptcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
cryptcfg.model_path = f'/{colab_path}/models/transformer/crypto_1021'
cryptdf = run_model_nn(f'/{colab_path}/tickers/cryptotickers.csv', cfg=cryptcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_close_model.pth'}


In [ ]:
cryptdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
15,XLM-USD,0.23,2025-11-21,2025-12-19,0.22,0.28,0.29,0.144928
6,DOGE-USD,0.14,2025-11-21,2025-12-19,0.14,0.16,0.18,0.142857
3,ADA-USD,0.41,2025-11-21,2025-12-19,0.39,0.48,0.53,0.138211
4,XRP-USD,1.96,2025-11-21,2025-12-19,1.93,2.34,2.39,0.132653
5,DOT-USD,2.37,2025-11-21,2025-12-19,2.25,3.10,2.64,0.123769
11,ICP-USD,4.36,2025-11-21,2025-12-19,3.64,4.84,5.74,0.087156
12,AVAX-USD,13.32,2025-11-21,2025-12-19,8.87,16.84,11.20,-0.076326
17,MATIC-USD,0.22,2025-03-24,2025-04-21,0.15,0.26,0.20,-0.075758
18,ATOM-USD,2.53,2025-11-21,2025-12-19,2.31,3.29,2.50,0.067194
16,AAVE-USD,157.80,2025-11-21,2025-12-19,152.88,185.84,162.92,0.059654


In [ ]:
etftcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
etftcfg.model_path = f'/{colab_path}/models/transformer/etf_1206'
etftdf = run_model_nn(f'/{colab_path}/tickers/etftickers.csv', cfg=etftcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/etf_1206_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/etf_1206_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/etf_1206_close_model.pth'}


In [ ]:
etftdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
19,URA,47.92,2025-12-05,2026-01-02,35.60,48.43,37.91,-0.151781
2,ARKW,155.82,2025-12-05,2026-01-02,130.61,163.09,130.07,-0.093463
0,ARKF,49.99,2025-12-05,2026-01-02,41.68,50.90,44.77,-0.084150
1,ARKK,81.91,2025-12-05,2026-01-02,67.07,87.07,71.94,-0.079966
17,SMH,364.42,2025-12-05,2026-01-02,299.18,378.65,329.92,-0.078216
7,IPO,47.59,2025-12-05,2026-01-02,41.49,49.11,43.71,-0.059256
9,IXN,106.51,2025-12-05,2026-01-02,92.00,109.05,100.04,-0.057710
32,XSW,191.70,2025-12-05,2026-01-02,170.91,195.70,178.99,-0.051295
27,XLP,78.46,2025-12-05,2026-01-02,78.07,84.04,82.45,0.039001
31,XME,99.74,2025-12-05,2026-01-02,92.36,103.15,92.30,-0.038132


In [ ]:
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
from google.auth import default


auth.authenticate_user()


creds, _ = default()
gc = gspread.authorize(creds)
# gc = gspread.authorize(GoogleCredentials.get_application_default())


In [ ]:
# Open your spreadsheet by name
spreadsheet = gc.open('Allocation')
# Or by URL: spreadsheet = gc.open_by_url('Your Spreadsheet URL')

# Select the worksheet (e.g., the first sheet)
worksheet = spreadsheet.worksheet("Buy")

# Get all values as a list of lists
data = worksheet.get_all_values()

# Convert to a Pandas DataFrame
sheetdf = pd.DataFrame(data[1:], columns=data[0]) # Assuming first row is header


In [ ]:
df = sheetdf[['Ticker']].copy()

# Apply the extraction function first, then filter based on the result
def extract_ticker(ticker_str):
    # Convert to string before splitting
    ticker_str = str(ticker_str)
    parts = ticker_str.split(':')
    if len(parts) > 1:
        return parts[1].strip() # Use strip to remove leading/trailing whitespace
    return ticker_str.strip() # Return the original string if no colon, also strip whitespace

df['Ticker'] = df['Ticker'].apply(extract_ticker)
# Remove BTCUSD and add BTC-USD, ETH-USD, ADA-USD, SOL-USD in Ticker
df = df[df['Ticker'] != 'BTCUSD']
df = pd.concat([df, pd.DataFrame({'Ticker': ['BTC-USD', 'ETH-USD', 'ADA-USD', 'SOL-USD']})], ignore_index=True)
df.to_csv(f'/{colab_path}/tickers/stocktickers.csv', index=False)
df.to_csv(f'/{colab_path}/tickers/buystockstickers.csv', index=False)

In [ ]:
# Only have Tickers column

run_model_nn('buystocks', '0402', False, 500, 250, copy_tickers=False).head(10)


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
333,QBTS,34.25,2025-10-08,2026-09-23,32.44,71.01,65.57,0.644964
96,CHGG,1.42,2025-10-08,2026-09-23,1.40,2.82,2.74,0.633803
97,SLP,16.47,2025-10-08,2026-09-23,16.09,32.07,31.69,0.616070
18,OUST,30.65,2025-10-08,2026-09-23,29.45,61.59,56.80,0.607830
9,HIMS,58.17,2025-10-08,2026-09-23,49.10,110.63,110.40,0.547934
252,LYFT,21.16,2025-10-08,2026-09-23,19.66,41.40,36.93,0.543636
6,DOMO,14.74,2025-10-08,2026-09-23,12.98,29.59,25.17,0.531886
326,ROOT,81.73,2025-10-08,2026-09-23,76.40,156.09,141.76,0.526367
122,LZ,9.97,2025-10-08,2026-09-23,9.63,19.32,16.65,0.524574
276,CRNC,11.96,2025-10-08,2026-09-23,11.62,21.43,21.43,0.518395


In [ ]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +180 -delete -print

sending incremental file list
buystocks_0709_predictions.csv

sent 18,897 bytes  received 35 bytes  37,864.00 bytes/sec
total size is 18,770  speedup is 0.99
